In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import CategoricalDtype

In [3]:
dtypes = pd.Series({'sepostulo': np.dtype('uint8'),'edad': np.dtype('uint16')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

trainingSet = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/TRAINING_SET_SIN_ENCODING.csv',dtype=column_types)
trainingSet.drop(['Unnamed: 0','denominacion_empresa'],axis=1,inplace=True)
trainingSet[["sexo",'nombre','estado','nombre_zona',
             'tipo_de_trabajo','nivel_laboral',
             'nombre_area']] = trainingSet[["sexo",'nombre','estado','nombre_zona',
                                            'tipo_de_trabajo','nivel_laboral',
                                            'nombre_area']].astype('category')

In [3]:
trainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9439964 entries, 0 to 9439963
Data columns (total 9 columns):
sepostulo          uint8
sexo               category
nombre             category
estado             category
edad               uint16
nombre_zona        category
tipo_de_trabajo    category
nivel_laboral      category
nombre_area        category
dtypes: category(7), uint16(1), uint8(1)
memory usage: 99.0 MB


# Me quedo con 500K de registros random

In [17]:
trainingSet_samples = trainingSet.sample(n=1000000,random_state=50)

In [11]:
trainingSet_samples['sepostulo'].value_counts()[0]

185383

# Hago dummificacion

In [18]:
trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.sexo))

trainingSet_samples.drop('sexo',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nombre))

trainingSet_samples.drop('nombre',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.estado))

trainingSet_samples.drop('estado',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nombre_zona))

trainingSet_samples.drop('nombre_zona',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.tipo_de_trabajo))

trainingSet_samples.drop('tipo_de_trabajo',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nivel_laboral,
                                                             prefix="Nivel_laboral"))

trainingSet_samples.drop('nivel_laboral',axis=1, inplace=True)

#trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nombre_area))
# LO SAQUE PORQUE MI MAQUINA NO SE BANCA POR AHI LA DE UDS SI AUNQUE AGREGA MUCHAS COLUMAS(188)

trainingSet_samples.drop('nombre_area',axis=1, inplace=True)

In [6]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

In [19]:
#Choose all predictors except target
from sklearn.model_selection import train_test_split
columnas = trainingSet_samples.columns.drop("sepostulo")
xtrain, xtest, ytrain, ytest = train_test_split(trainingSet_samples[columnas], 
                                        trainingSet_samples['sepostulo'],test_size=0.30, random_state=12)
casos_negativos = trainingSet_samples['sepostulo'].value_counts()[0]
casos_positivos = trainingSet_samples['sepostulo'].value_counts()[1]
scale_weight =  casos_negativos / casos_positivos

xgb_model = XGBClassifier(
 learning_rate =0.1,
 n_estimators=300,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=1,
 scale_pos_weight= scale_weight,
 seed=27)

# Train and evaluate.
xgb_model.fit(xtrain, ytrain, eval_metric=['error'], eval_set=[((xtrain, ytrain)),(xtest, ytest)])

from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

y_pred = xgb_model.predict(xtest)  #predicts
print('confusion matrix')
print(metrics.confusion_matrix(ytest, y_pred))
print('classification report')
print(metrics.classification_report(ytest, y_pred))
print("-----------------------------------------------------------------------------------------")
print("Accuracy is :")
print(metrics.accuracy_score(ytest, y_pred))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, y_pred)))

[0]	validation_0-error:0.485	validation_1-error:0.485017
[1]	validation_0-error:0.485301	validation_1-error:0.485307
[2]	validation_0-error:0.49657	validation_1-error:0.49644
[3]	validation_0-error:0.486161	validation_1-error:0.485953
[4]	validation_0-error:0.484356	validation_1-error:0.48435
[5]	validation_0-error:0.479236	validation_1-error:0.479587
[6]	validation_0-error:0.485511	validation_1-error:0.48564
[7]	validation_0-error:0.485544	validation_1-error:0.485663
[8]	validation_0-error:0.48456	validation_1-error:0.48456
[9]	validation_0-error:0.480703	validation_1-error:0.481047
[10]	validation_0-error:0.48072	validation_1-error:0.48099
[11]	validation_0-error:0.48076	validation_1-error:0.481037
[12]	validation_0-error:0.480384	validation_1-error:0.48065
[13]	validation_0-error:0.480133	validation_1-error:0.48054
[14]	validation_0-error:0.480187	validation_1-error:0.480617
[15]	validation_0-error:0.480903	validation_1-error:0.481373
[16]	validation_0-error:0.4811	validation_1-erro

In [20]:
casos_negativos = trainingSet_samples['sepostulo'].value_counts()[1]
casos_positivos = trainingSet_samples['sepostulo'].value_counts()[0]
scale_weight =  casos_negativos / casos_positivos

xgb_model = XGBClassifier(
 learning_rate =0.1,
 n_estimators=300,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=1,
 scale_pos_weight= scale_weight,
 seed=27)

# Train and evaluate.
xgb_model.fit(xtrain, ytrain, eval_metric=['error'], eval_set=[((xtrain, ytrain)),(xtest, ytest)])

from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

y_pred = xgb_model.predict(xtest)  #predicts
print('confusion matrix')
print(metrics.confusion_matrix(ytest, y_pred))
print('classification report')
print(metrics.classification_report(ytest, y_pred))
print("-----------------------------------------------------------------------------------------")
print("Accuracy is :")
print(metrics.accuracy_score(ytest, y_pred))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, y_pred)))

[0]	validation_0-error:0.371286	validation_1-error:0.370057
[1]	validation_0-error:0.371241	validation_1-error:0.370073
[2]	validation_0-error:0.371111	validation_1-error:0.369953
[3]	validation_0-error:0.371241	validation_1-error:0.370047
[4]	validation_0-error:0.371074	validation_1-error:0.369923
[5]	validation_0-error:0.371003	validation_1-error:0.369867
[6]	validation_0-error:0.370983	validation_1-error:0.369907
[7]	validation_0-error:0.371029	validation_1-error:0.369917
[8]	validation_0-error:0.370999	validation_1-error:0.369903
[9]	validation_0-error:0.371023	validation_1-error:0.369923
[10]	validation_0-error:0.371069	validation_1-error:0.36996
[11]	validation_0-error:0.371149	validation_1-error:0.37003
[12]	validation_0-error:0.371017	validation_1-error:0.36995
[13]	validation_0-error:0.370963	validation_1-error:0.369907
[14]	validation_0-error:0.370973	validation_1-error:0.369913
[15]	validation_0-error:0.370969	validation_1-error:0.369913
[16]	validation_0-error:0.370923	vali

In [11]:
#Choose all predictors except target
target = 'sepostulo'
scale_weight = trainingSet_samples['sepostulo'].value_counts()[0] / 
            trainingSet_samples['sepostulo'].value_counts()[1]
predictors = [x for x in trainingSet_samples.columns if x not in [target]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=300,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=1,
 scale_pos_weight= scale_weight,
 seed=27)
modelfit(xgb1, trainingSet_samples, predictors)

    
cv_folds=5
early_stopping_rounds=50
xgb_param = xgb1.get_xgb_params()
xgtrain = xgb.DMatrix(trainingSet_samples[predictors].values, label=trainingSet_samples[target].values)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb1.get_params()['n_estimators'],
                  nfold=cv_folds, metrics={'auc'}, early_stopping_rounds=early_stopping_rounds)
xgb1.set_params(n_estimators=cvresult.shape[0])

#Fit the algorithm on the data
xgb1.fit(trainingSet_samples[predictors], trainingSet_samples['sepostulo'],eval_metric='auc')

#Predict training set:
dtrain_predictions = xgb1.predict(trainingSet_samples[predictors])
dtrain_predprob = xgb1.predict_proba(trainingSet_samples[predictors])[:,1]

#Print model report:
print ("\nModel Report")
print ("Accuracy : %.4g" % metrics.accuracy_score(trainingSet_samples['sepostulo'].values, dtrain_predictions))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(trainingSet_samples['sepostulo'], dtrain_predprob))

/home/lucio/Documentos/Datos/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 0.6354
AUC Score (Train): 0.569637


TypeError: 'str' object is not callable

# TestingSet

In [21]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], 
                                               ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado',
                            'Secundario','Terciario/Técnico', 'Universitario'],ordered=False),
                    'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

testingSet_CON_NAN = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/TestingSets/testingSet_CON_NAN.csv',dtype=column_types)
testingSet_CON_NAN.drop('Unnamed: 0',inplace=True,axis=1)
testingSet_CON_NAN.drop(columns=['idpostulante','denominacion_empresa','nombre_area'],inplace=True) #No sirve para el algoritmo de ML

In [29]:
testingSet_CON_NAN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
nombre_zona        100000 non-null object
tipo_de_trabajo    100000 non-null object
nivel_laboral      100000 non-null object
sexo               93375 non-null category
nombre             93375 non-null category
estado             93375 non-null category
edad               93375 non-null float64
dtypes: category(3), float64(1), int64(2), object(3)
memory usage: 4.9+ MB


# Aplico dummies

In [22]:
testingSet_CON_NAN = testingSet_CON_NAN.join(pd.get_dummies(testingSet_CON_NAN.sexo))

testingSet_CON_NAN.drop('sexo',axis=1, inplace=True)

testingSet_CON_NAN = testingSet_CON_NAN.join(pd.get_dummies(testingSet_CON_NAN.nombre))

testingSet_CON_NAN.drop('nombre',axis=1, inplace=True)

testingSet_CON_NAN = testingSet_CON_NAN.join(pd.get_dummies(testingSet_CON_NAN.estado))

testingSet_CON_NAN.drop('estado',axis=1, inplace=True)

testingSet_CON_NAN = testingSet_CON_NAN.join(pd.get_dummies(testingSet_CON_NAN.nombre_zona))

testingSet_CON_NAN.drop('nombre_zona',axis=1, inplace=True)

testingSet_CON_NAN = testingSet_CON_NAN.join(pd.get_dummies(testingSet_CON_NAN.tipo_de_trabajo))

testingSet_CON_NAN.drop('tipo_de_trabajo',axis=1, inplace=True)

testingSet_CON_NAN = testingSet_CON_NAN.join(pd.get_dummies(testingSet_CON_NAN.nivel_laboral,
                                                             prefix="Nivel_laboral"))

testingSet_CON_NAN.drop('nivel_laboral',axis=1, inplace=True)

#testingSet_CON_NAN = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nombre_area))

In [23]:
# Get missing columns in the training test
missing_cols = set( trainingSet_samples.columns ) - set( testingSet_CON_NAN.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    testingSet_CON_NAN[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
testingSet_CON_NAN = testingSet_CON_NAN[trainingSet_samples.columns]

In [24]:
#Predict testing set:
dtest_predictions = xgb_model.predict(testingSet_CON_NAN[predictors])
dtest_predprob = xgb_model.predict_proba(testingSet_CON_NAN[predictors])[:,1]

In [25]:
testingSet_CON_NAN['sepostulo'] = dtest_predictions

In [26]:
testingSet_CON_NAN['sepostulo'].value_counts()

1    99136
0      864
Name: sepostulo, dtype: int64

In [45]:
testingSet_CON_NAN.head()

,sepostulo,edad,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,Secundario,...,Por Horas,Primer empleo,Teletrabajo,Temporario,Voluntario,Nivel_laboral_Gerencia / Alta Gerencia / Dirección,Nivel_laboral_Jefe / Supervisor / Responsable,Nivel_laboral_Junior,Nivel_laboral_Otro,Nivel_laboral_Senior / Semi-Senior
0,1,41.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,30.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,35.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,68.0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1,31.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [47]:
submit = testingSet_CON_NAN[['sepostulo']]
submit.head()

,sepostulo
0,1
1,1
2,1
3,1
4,1


In [49]:
submit.insert(0, 'id', testingSet_CON_NAN['id'])

In [51]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
id           100000 non-null int64
sepostulo    100000 non-null uint8
dtypes: int64(1), uint8(1)
memory usage: 879.0 KB


In [52]:
submit.to_csv('Submit/prediccion20_XGBOOST_500K_dummies.csv',index=False)

# Tuning max_depth y min_child_weight

In [53]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch1.fit(trainingSet_samples[predictors],trainingSet_samples[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/home/lucio/Documentos/Datos/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.55301, std: 0.00136, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.55301, std: 0.00133, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.55313, std: 0.00140, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.55625, std: 0.00137, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.55651, std: 0.00128, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.55644, std: 0.00130, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.55807, std: 0.00125, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.55772, std: 0.00102, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.55789, std: 0.00118, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.55784, std: 0.00153, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.55783, std: 0.00124, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.55780, std: 0.00130, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 7, 'min_child_weight': 1

In [55]:
param_test2 = {
 'max_depth':[6,7,8],
 'min_child_weight':[1,2]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch2.fit(trainingSet_samples[predictors],trainingSet_samples[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/home/lucio/Documentos/Datos/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.55746, std: 0.00144, params: {'max_depth': 6, 'min_child_weight': 1},
  mean: 0.55753, std: 0.00147, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.55807, std: 0.00125, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.55799, std: 0.00134, params: {'max_depth': 7, 'min_child_weight': 2},
  mean: 0.55824, std: 0.00123, params: {'max_depth': 8, 'min_child_weight': 1},
  mean: 0.55793, std: 0.00106, params: {'max_depth': 8, 'min_child_weight': 2}],
 {'max_depth': 8, 'min_child_weight': 1},
 0.5582404177404134)

# Tuning Gamma

In [56]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=1,iid=False, cv=5)
gsearch3.fit(trainingSet_samples[predictors],trainingSet_samples[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/home/lucio/Documentos/Datos/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.55824, std: 0.00123, params: {'gamma': 0.0},
  mean: 0.55810, std: 0.00134, params: {'gamma': 0.1},
  mean: 0.55797, std: 0.00132, params: {'gamma': 0.2},
  mean: 0.55796, std: 0.00145, params: {'gamma': 0.3},
  mean: 0.55802, std: 0.00116, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.5582404177404134)

In [60]:
#Predict testing set:
dtest_predictions = gsearch3.predict(testingSet_CON_NAN[predictors])
dtest_predprob = gsearch3.predict_proba(testingSet_CON_NAN[predictors])[:,1]

/home/lucio/Documentos/Datos/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
testingSet_CON_NAN['sepostulo'] = dtest_predictions
submit = testingSet_CON_NAN[['sepostulo']]
submit.head()

,sepostulo
0,1
1,1
2,1
3,1
4,1


In [63]:
submit.insert(0, 'id', testingSet_CON_NAN['id'])

In [64]:
submit['sepostulo'].value_counts()

1    91351
0     8649
Name: sepostulo, dtype: int64

In [65]:
submit.to_csv('Submit/prediccion23_XGBOOST_500K_dummies_algo_tuning.csv',index=False)